In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
%matplotlib inline
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torchmetrics
from sklearn.model_selection import train_test_split

In [111]:
import transformers

In [30]:
dataset_hugging_face = load_dataset("ucberkeley-dlab/measuring-hate-speech", 'binary')
df_train = dataset_hugging_face['train'].to_pandas()
def data_from_hugging_face(df):
    feature = [
        'hate_speech_score',
        'text',
        'target_race',
        'target_religion',
        'target_gender'
    ]
    df = df[feature]
    new_df = df[(df['target_race'] == True) | (df['target_religion'] == True) | (df['target_gender'] == True)]

    return new_df
df = data_from_hugging_face(df_train)



Using custom data configuration ucberkeley-dlab--measuring-hate-speech-c32713cabe528196
Found cached dataset parquet (/Users/pierredemetz/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [110]:
df.to_pickle('df_hate.pkl')

In [46]:
train_df, test_df = train_test_split(df, test_size=0.1)

In [47]:
train_df.iloc

In [48]:
Classes = ['hate_speech_score', 'target_race', 'target_religion', 'target_gender']

In [49]:
BERT_MODEL_NAME = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [67]:
class ToxicCommentsDataset:#(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_len: int = 128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        single_row = self.data.iloc[index]
        
        comment = single_row['text']
        labels = single_row[Classes]
        labels[['target_race', 'target_religion', 'target_gender']] = labels[['target_race', 'target_religion', 'target_gender']].astype(int)

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        return {
            "comment_text": comment,
            "input_ids": encoding["input_ids"].flatten(), # [1,512] => [512]
            "attention_mask": encoding["attention_mask"].flatten(), # [1,512] => [512]
            "labels": torch.FloatTensor(labels)
        }

In [96]:
train_dataset = ToxicCommentsDataset(train_df, tokenizer)

In [97]:
train_dataset

In [104]:
train_dataset[30]

{'comment_text': "The bodies of dead Terrorists like Zakir Musa etc should be dumped in sewer from next time. This Terrorist's father must be booked & put in Jail. These supporters, soon to be Terrorists must be eliminated from the valley. @PMOIndia #Kashmir #RemoveArticle370and35A",
 'input_ids': tensor([  101,  1109,  3470,  1104,  2044, 15618,  3681,  1176,   163,  8552,
          1197, 27389,  3576,  1431,  1129, 14632,  1107, 27635,  1121,  1397,
          1159,   119,  1188, 15618,  1776,   112,   188,  1401,  1538,  1129,
         18951,   111,  1508,  1107, 19183,  1233,   119,  1636,  6638,   117,
          1770,  1106,  1129, 15618,  3681,  1538,  1129,  5802,  1121,  1103,
          4524,   119,   137, 14123,  2346,  2240, 12090,  1161,   108, 13705,
           108, 11336,  3702,  2707,  1592,  3740, 26726, 26303,  1568,  5709,
         19297,  1592,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0

In [100]:
class ToxicCommentDataModule(pl.LightningDataModule):
    def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_len=128):
        super().__init__()
        
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_len = max_len
    
    def setup(self, stage=None):
        self.train_dataset = ToxicCommentsDataset(self.train_df, self.tokenizer, self.max_len)
        self.test_dataset = ToxicCommentsDataset(self.test_df, self.tokenizer, self.max_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True,num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.test_dataset,batch_size=1,num_workers=4)    
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset,batch_size=1,num_workers=4)

In [101]:
EPOCHS = 2
BATCH_SIZE = 32

data_module = ToxicCommentDataModule(
    train_df, 
    test_df, 
    tokenizer, 
    batch_size=BATCH_SIZE
)



data_module.setup() 

In [107]:
type(data_module)

__main__.ToxicCommentDataModule

In [102]:
train_dataset

In [91]:
class ToxicCommentClassifier(pl.LightningModule):
    def __init__(self, n_classes: int, steps_per_epoch=None, n_epochs=None):
        super().__init__()

        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.criterion = nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {
            "loss": loss,
            "predictions": outputs,
            "labels": labels
        }

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def training_epoch_end(self, outputs):
        labels = []
        predictions = []
        
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)

            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        print("#####")
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)

        for i, name in enumerate(Classes):
            roc_score = torchmetrics.AUROC(predictions[:, i], labels[:, i])
            self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", roc_score, self.current_epoch)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5)
        warmup_steps = self.steps_per_epoch // 3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps
        scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer], [scheduler]

In [92]:
model = ToxicCommentClassifier(
    n_classes=len(Classes), 
    steps_per_epoch=len(train_df)//BATCH_SIZE, 
    n_epochs=EPOCHS
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [108]:
trainer = pl.Trainer(max_epochs=EPOCHS)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [112]:
trainer.fit(model, data_module)

/Users/pierredemetz/.pyenv/versions/ucl/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 108 M 
1 | classifier | Linear    | 3.1 K 
2 | criterion  | BCELoss   | 0     
-----------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.253   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/pierredemetz/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/pierredemetz/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'ToxicCommentsDataset' on <module '__main__' (built-in)>
/Users/pierredemetz/.pyenv/versions/ucl/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
